# PyGMTSAR Python Notebook S1A_Stack_CPGF_T173

```
It works on

* MacOS Catalina (Python 3.9) - please pre-install system dependencies (maybe using HomeBrew)

* Google Cloud Notebooks on Debian 10 (Python 3.7) - use the cloud init script from my repository https://github.com/mobigroup/gmtsar/blob/master/gmtsar/sh/GMTSAR.install.debian10.sh

* Google Colab (Python 3.7) - you will be asked to re-run the notebook once using menu "Kernel" -> "Restart and Run All". Note: to open all notebook cells select menu "View" -> "Expand Sections"
```

> ### I'm a freelancer and that's my free time Open Source project with GPL-3.0 License. You are able to sponsor my projects on [Patreon: Become a Patron!](https://www.patreon.com/bePatron?u=54500608) and order research, development and support on [Upwork](https://www.upwork.com/freelancers/~01e65e8e7221758623)
>
> ### @ Alexey Pechnikov, Sep, 2021
> 
> [Geological models on YouTube channel](https://www.youtube.com/channel/UCSEeXKAn9f_bDiTjT6l87Lg)
>
> [Augmented Reality (AR) Geological Models](https://mobigroup.github.io/ParaView-Blender-AR/)
>
> [GitHub repositories](https://github.com/mobigroup)
>
> [English posts and articles](https://www.linkedin.com/in/alexey-pechnikov/)
>
> [Russian articles](https://habr.com/ru/users/N-Cube/posts/)

## Debian 10 and Google Colab GMTSAR Installation

### On Google Cloud AI Notebooks: check root access

```On Google Cloud AI Notebooks sometimes we have an issue when "sudo" requires a password. In this case drop the instance and create a new one and - that's important - wait 5-10 minutes before connect to it using link "OPEN JUPYTERLAB"```

In [ ]:
uname = !uname
if uname == ['Linux']:
    !sudo date

### Install https://github.com/mobigroup/gmtsar

In [ ]:
uname = !uname
count = !ls /usr/local | grep GMTSAR | wc -l
if uname == ['Linux'] and count == ['0']:
    !apt install -y csh autoconf gfortran \
        libtiff5-dev libhdf5-dev liblapack-dev libgmt-dev gmt-dcw gmt-gshhg gmt > /dev/null
    !cd /usr/local && git clone --branch master https://github.com/mobigroup/gmtsar GMTSAR > /dev/null
    !cd /usr/local/GMTSAR && autoconf > /dev/null
    !cd /usr/local/GMTSAR && ./configure --with-orbits-dir=/tmp > /dev/null
    !cd /usr/local/GMTSAR && make 1>/dev/null 2>/dev/null
    !cd /usr/local/GMTSAR && make install >/dev/null

## Download and unpack the example, create processing directory

In [ ]:
count = !ls | grep S1A_Stack_CPGF_T173.tar.gz | wc -l
if count == ['0']:
    print ('Downloading the example...')
    !wget -c http://topex.ucsd.edu/gmtsar/tar/S1A_Stack_CPGF_T173.tar.gz
    !tar xvzf S1A_Stack_CPGF_T173.tar.gz -C .

# optionally recreate processing directory for fresh run
!rm -rf raw
!mkdir -p raw

## Define ENV Variables for Jupyter Instance

In [ ]:
import os

# use default GMTSAR installation path
GMTSAR = '/usr/local/GMTSAR'
PATH = os.environ['PATH']

if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + f':{GMTSAR}/bin/'
    %env PATH {PATH}
    %env GMTSAR {GMTSAR}

## Install Python Modules

```Maybe you need to restart your notebook, follow the instructions printing below```

```The installation takes a long time on fresh Debian 10 and a short time on Google Colab```

In [ ]:
import sys
!{sys.executable} --version

In [ ]:
#!{sys.executable} -m pip install --upgrade pip setuptools wheel 1>/dev/null 2>/dev/null
!{sys.executable} -m pip install cartopy==0.19.0.post1 1>/dev/null 2>/dev/null
!{sys.executable} -m pip install xarray==0.19.0        1>/dev/null 2>/dev/null
!{sys.executable} -m pip install scipy==1.7.1          1>/dev/null 2>/dev/null

In [ ]:
!{sys.executable} -m pip install \
    h5py netcdf4 h5netcdf \
    rasterio rioxarray numpy \
    scikit-image scipy sklearn \
    xarray dask distributed zarr \
    pandas geopandas \
    sentineleof elevation \
    matplotlib seaborn geoviews hvplot datashader bokeh \
    xmltodict joblib tqdm 1>/dev/null 2>/dev/null

### On Google Colab: kernel restart required once
```On Google Colab we need to restart kernel once when all the modules installed```

In [ ]:
import xarray
print (xarray.__version__)
assert xarray.__version__ == '0.19.0', """
***********************************************************************************
*
Do not worry, just restart the notebook kernel once (Kernel -> Restart and Run All)
*
***********************************************************************************
"""

## Load and Setup Python Modules

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
# supress numpy warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# plotting modules
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
import holoviews as hv
from holoviews import opts
from bokeh.models import FixedTicker
hv.extension('bokeh', 'matplotlib')
#pd.options.plotting.backend = 'holoviews'
pd.options.plotting.backend = 'hvplot'
import seaborn as sns
import cartopy.crs as ccrs
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [ ]:
# define Pandas display settings
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

gstiles = hv.Tiles('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name='Google Satellite')
ottiles = hv.Tiles('https://tile.opentopomap.org/{Z}/{X}/{Y}.png', name='Open Topo')

## Load Custom Python Modules

In [ ]:
sys.path.append(os.path.join(os.environ['GMTSAR'],'gmtsar', 'py'))

from PRM import PRM
from SBAS import SBAS

## Define Parameters

In [ ]:
MASTER       = '2015-04-03'
WORKDIR      = 'raw'
DATADIR      = 'raw_orig'
DEMFILE      = 'topo/dem.grd'
BASEDAYS     = 100
BASEMETERS   = 150
CORRLIMIT    = 0.10
DEFOMAX      = 0

## Init SBAS

In [ ]:
sbas = SBAS(DATADIR, DEMFILE, WORKDIR).set_master(MASTER)
sbas.to_dataframe()

### Static Plots

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
sbas.get_dem()[::4,::4].plot.imshow(cmap='Blues_r', vmin=0)
plt.scatter(sbas.geoloc()['longitude'], sbas.geoloc()['latitude'], c=sbas.geoloc()['pixel'], cmap='jet')
plt.title('Sentinel1 Frame on DEM plus GCP', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
sbas.get_dem(geoloc=True)[::4,::4].plot.imshow(cmap='Blues_r', vmin=0)
plt.scatter(sbas.geoloc()['longitude'], sbas.geoloc()['latitude'], c=sbas.geoloc()['pixel'], cmap='jet')
plt.title('Sentinel1 Frame DEM cropped using GCP', fontsize=18)
plt.show()

### Interactive Plots

```Be careful because these plots require more RAM to be visualized```

## Stack Images (for a single subswath only)

In [ ]:
sbas.stack_parallel()

## SBAS Baseline

In [ ]:
baseline_pairs = sbas.baseline_pairs(days=BASEDAYS, meters=BASEMETERS)
baseline_pairs

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
ax = plt.gca()

lines = [[(row.ref_timeline,row.ref_baseline),(row.rep_timeline,row.rep_baseline)]
         for row in baseline_pairs.itertuples()]
lc = matplotlib.collections.LineCollection(lines, colors='#30a2da', linewidths=1.5)
ax.add_collection(lc)
ax.autoscale()
ax.margins(0.5)
bs1 = baseline_pairs[['ref_timeline','ref_baseline','ref_date']].values
bs2 = baseline_pairs[['rep_timeline','rep_baseline','rep_date']].values
df = pd.DataFrame(np.concatenate([bs1, bs2]), columns=['timeline','baseline','date']).drop_duplicates()
for x,y,label in df.values:
    plt.annotate(label, (x,y), textcoords="offset points", xytext=(40,5), ha='center')
ax.set_xlabel('Timeline', fontsize=16)
ax.set_ylabel('Perpendicular Baseline, [m]', fontsize=16)
ax.set_title('SBAS Baseline', fontsize=18)
plt.show()

## DEM in Radar Coordinates

In [ ]:
%%time

sbas.topo_ra()

### Load Grids

In [ ]:
topo_ra = xr.open_dataarray(f'{WORKDIR}/topo_ra.grd')

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
topo_ra[::4,::4].plot.imshow(cmap='Blues_r', vmin=0)
plt.xlabel('Range', fontsize=16)
plt.ylabel('Azimuth', fontsize=16)
plt.title('Topography in Radar Coordinates', fontsize=18)
plt.show()

### Interactive Plots

```Be careful because these plots require more RAM to be visualized```

## Interferograms

In [ ]:
pairs = baseline_pairs[['ref_date', 'rep_date']]
pairs

In [ ]:
# we can just miss "func" argument when post-processing is not required
# define a postprocessing function for decimation, etc.
decimator = lambda dataarray: dataarray.coarsen({'y': 4, 'x': 4}, boundary='trim').mean()

# default parameters: wavelength=200, psize=32, func=None (no postprocessing)
sbas.intf_parallel(pairs, wavelength=400, func=decimator)

### Load Grids

In [ ]:
# TODO: add geocode=True/False argument
phasefilts = sbas.open_grids(pairs, 'phasefilt')

### Static Plots

In [ ]:
fg = phasefilts.plot.imshow(
    col="pair",
    col_wrap=3, size=4, aspect=1.2,
    vmin=-np.pi, vmax=np.pi, cmap='gist_rainbow_r'
)
fg.set_axis_labels(x_var='Range', y_var='Azimuth')
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('Filtered Phase, [rad]', y=1.05, fontsize=24)
plt.show()

### Interactive Plots

```Be careful because these plots require more RAM to be visualized```

### Load Correlation Grids

In [ ]:
corrs = sbas.open_grids(pairs, 'corr')

### Static Plots

In [ ]:
fg = corrs.plot.imshow(
    col="pair",
    col_wrap=3, size=4, aspect=1.2,
    clim=(0, 0.8), cmap='gray'
)
fg.set_axis_labels(x_var='Range', y_var='Azimuth')
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('Correlation', y=1.05, fontsize=24)
plt.show()

### Interactive Plots

```Be careful because these plots require more RAM to be visualized```

## Unwrapping

In [ ]:
# generate a custom snaphu config file and use it as argument "conf" value
# conf = self.PRM().snaphu_config(defomax=0)

# we can just miss "func" argument when post-processing is not required
# define a post-processing function to crop and interpolate low-coherence areas, etc.
cleaner = lambda corr, unwrap: xr.where(corr>=CORRLIMIT, unwrap, np.nan)
#cleaner = lambda corr, unwrap: sbas.nearest_grid(xr.where(corr>=CORRLIMIT, unwrap, np.nan))

# default parameters: threshold=0.1, conf=None, func=None (no postprocessing required)
sbas.unwrap_parallel(pairs, threshold=CORRLIMIT, func=cleaner)

### Load Grids

In [ ]:
unwraps = sbas.open_grids(pairs, 'unwrap')

### Static Plots

In [ ]:
zmin, zmax = np.nanquantile(unwraps, [0.01, 0.99])
fg = unwraps.plot.imshow(
    col="pair",
    col_wrap=3, size=4, aspect=1.2,
    vmin=zmin, vmax=zmax, cmap='jet'
)
fg.set_axis_labels(x_var='Range', y_var='Azimuth')
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('Unwrapped Phase, [rad]', y=1.05, fontsize=24)
plt.show()

### Interactive Plots

```Be careful because these plots require more RAM to be visualized```

## LOS Displacement

### Calculate Grids

In [ ]:
los_disp_mm = sbas.open_grids(pairs, 'unwrap', func=sbas.los_displacement_mm)

### Static Plots

In [ ]:
zmin, zmax = np.nanquantile(los_disp_mm, [0.01, 0.99])
fg = los_disp_mm.plot.imshow(
    col="pair",
    col_wrap=3, size=4, aspect=1.2,
    vmin=zmin, vmax=zmax, cmap='jet'
)
fg.set_axis_labels(x_var='Range', y_var='Azimuth')
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('LOS Displacement, [mm]', y=1.05, fontsize=24)
plt.show()

### Interactive Plots

```Be careful because these plots require more RAM to be visualized```

### Geocoding

In [ ]:
los_disp_mm_ll = sbas.open_grids(pairs, 'unwrap', geocode=True, func=sbas.los_displacement_mm)

### Static Plots

In [ ]:
zmin, zmax = np.nanquantile(los_disp_mm_ll, [0.01, 0.99])
fg = los_disp_mm_ll.plot.imshow(
    col="pair",
    col_wrap=3, size=4, aspect=1.2,
    vmin=zmin, vmax=zmax, cmap='jet'
)
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('LOS Displacement in Geographic Coordinates, [mm]', y=1.05, fontsize=24)
plt.show()

## Bonus: Inverted Interferograms

```We are able to build reverse-ordered interferograms by a simple hand move```

### Load Grids

### Static Plots

### Interactive Plots

```Be careful because these plots require more RAM to be visualized```

## Bonus: Inverted Interferogram Unwrapping

### Load Grids

### Static Plots

### Interactive Plots

```Be careful because these plots require more RAM to be visualized```

## Bonus: Interferogram Unwrapping Two-Ways Difference

### Calculate Grids

### Static Plots

### Interactive Plots

```Be careful because these plots require more RAM to be visualized```

## SBAS Velocity

In [ ]:
sbas.sbas(baseline_pairs, smooth=1)

### Load Grids with Interpolation and Geocoding

In [ ]:
vel = sbas.open_grid('vel')
vel_ll = sbas.open_grid('vel', geocode=True)
# interpolate low-coherency areas and mask not valid pixels
vel_filled = sbas.open_grid('vel', func=sbas.nearest_grid, mask=True)
vel_filled_ll = sbas.open_grid('vel', geocode=True, func=sbas.nearest_grid, mask=True)

### Static Plots

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8), dpi=300)
zmin, zmax = np.nanquantile(vel, [0, 0.99])
vel.plot(vmin=zmin, vmax=zmax, cmap='jet', ax=ax1)
zmin, zmax = np.nanquantile(vel_ll, [0, 0.99])
vel_ll.plot(vmin=zmin, vmax=zmax, cmap='jet', ax=ax2)
zmin, zmax = np.nanquantile(vel_filled, [0, 0.99])
vel_filled.plot(vmin=zmin, vmax=zmax, cmap='jet', ax=ax3)
zmin, zmax = np.nanquantile(vel_filled_ll, [0, 0.99])
vel_filled_ll.plot(vmin=zmin, vmax=zmax, cmap='jet', ax=ax4)
ax1.set_title('Radar Coordinates', fontsize=16)
ax2.set_title('Geographic Coordinates', fontsize=16)
plt.suptitle('SBAS Velocity, [mm/year]', fontsize=18)
plt.show()

### Interactive Plots

```Be careful because these plots require more RAM to be visualized```

```The plots below do not work on Debian 10 and Python 3.7```

In [ ]:
uname = !uname
plots = None
if uname == ['Darwin']:
    zmin, zmax = np.nanquantile(vel_filled_ll, [0, 0.99])
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':330, 'height':280,
                   'cmap':'jet', 'colorbar':True, 'clim':(zmin, zmax)}
    opts1 = {'tiles':gstiles, 'alpha':0.4, 'title':'Google Satellite'}
    opts2 = {'tiles':ottiles, 'alpha':0.4, 'title':'OpenTopoMap'}
    opts3 = {'tiles':False,   'alpha':1.4, 'title':'SBAS Velocity'}
    plots = vel_filled_ll.hvplot(**{**opts_common, **opts1}).opts(xrotation=45) + \
    vel_filled_ll.hvplot(**{**opts_common, **opts2}).opts(xrotation=45) + \
    vel_filled_ll.hvplot(**{**opts_common, **opts3}).opts(xrotation=45)
plots